In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, MaxPooling2D
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add, LeakyReLU, Lambda
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D,Conv2DTranspose
from keras.applications import VGG19,inception_v3
from keras.models import Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
from helper import DataLoader
import numpy as np
import os
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot
from keras.callbacks import TensorBoard, ModelCheckpoint
from glob import glob
from tqdm import tqdm_notebook
import scipy
import imageio
from tensorlayer.prepro import *
import tensorlayer as tl
from keras import initializers
from keras import backend as K
import tensorflow as tf
import h5py

In [3]:
batch_size = 16
optimizer = Adam(lr=0.001, beta_1=0.9)
ni = np.sqrt(batch_size)
hr_shape = (224,224,3)
lr_shape = (56,56,3)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def SubpixelConv2D(input_shape, scale=4):
    """
    Keras layer to do subpixel convolution.
    NOTE: Tensorflow backend only. Uses tf.depth_to_space
    Ref:
        [1] Real-Time Single Image and Video Super-Resolution Using an Efficient Sub-Pixel Convolutional Neural Network
            Shi et Al.
            https://arxiv.org/abs/1609.05158
    :param input_shape: tensor shape, (batch, height, width, channel)
    :param scale: upsampling scale. Default=4
    :return:
    """
    # upsample using depth_to_space
    def subpixel_shape(input_shape):
        dims = [input_shape[0],
                input_shape[1] * scale,
                input_shape[2] * scale,
                int(input_shape[3] / (scale ** 2))]
        output_shape = tuple(dims)
        return output_shape

    def subpixel(x):
        return tf.depth_to_space(x, scale)


    return Lambda(subpixel, output_shape=subpixel_shape)

In [5]:
def Generator(lr_shape,is_train=True):
    w_init = initializers.RandomNormal(stddev=0.02, seed=None)
    g_init = initializers.RandomNormal(mean=0.0,stddev=0.02, seed=None)
    lr_input = Input(shape=lr_shape,name="in")
    n = Conv2D(64, kernel_size=9, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c')(lr_input)
    temp = n
    # Res blocks
    for i in range(16):
        nn = Conv2D(64, (3, 3), strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c1/%s' % i)(n)
        if is_train: 
            nn = BatchNormalization(gamma_initializer=g_init, name='n64s1/b1/%s' % i)(nn)
            nn = Activation('relu')(nn)
        nn = Conv2D(64, (3, 3), strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c2/%s' % i)(nn)
        if is_train: 
            nn = BatchNormalization(gamma_initializer=g_init, name='n64s1/b2/%s' % i)(nn)
        nn = Add(name='b_residual_add/%s' % i)([n, nn])
        n = nn

    n = Conv2D(64, (3, 3), strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c/m')(n)
    if is_train: n = BatchNormalization(gamma_initializer=g_init, name='n64s1/b/m')(n)
    n = Add(name='add3')([n, temp])
    # End of Res Blocks

    n = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')(n)
    pixelShuffle1 = SubpixelConv2D((None, 56, 56, 256),scale=2)(n)
    if is_train: pixelShuffle1 = BatchNormalization(gamma_initializer=g_init)(pixelShuffle1)
    pixelShuffle1 = Activation('relu')(pixelShuffle1)

    pixelShuffle2 = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')((pixelShuffle1))
    pixelShuffle2 = SubpixelConv2D((None, 112, 112, 256),scale=2)(pixelShuffle2)
    if is_train: pixelShuffle2 = BatchNormalization(gamma_initializer=g_init)(pixelShuffle2)
    pixelShuffle2 = Activation('relu')(pixelShuffle2)

    out = Conv2D(3, (9, 9),strides=1, activation='tanh', padding='same', kernel_initializer=w_init, name='out')(pixelShuffle2)
    return Model(lr_input,out)

SRGAN_gen = Generator(lr_shape)
SRGAN_gen.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
in (InputLayer)                 (None, 56, 56, 3)    0                                            
__________________________________________________________________________________________________
n64s1/c (Conv2D)                (None, 56, 56, 64)   15616       in[0][0]                         
__________________________________________________________________________________________________
n64s1/c1/0 (Conv2D)             (None, 56, 56, 64)   36928       n64s1/c[0][0]                    
__________________________________________________________________________________________________
n64s1/b1/0 (BatchNormalization) (None, 56, 56, 64)   256         n64s1/c1/0[0][0]                 
__________________________________________________________________________________________________
activation

In [6]:
def Discriminator(hr_shape,is_train=True):
    w_init = initializers.RandomNormal(stddev=0.02, seed=None)
    gamma_init = initializers.RandomNormal(mean=1,stddev=0.02, seed=None)
    inputLayer = Input(hr_shape, name='in')
    n = Conv2D(64, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c')(inputLayer)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*2, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n128s1/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*2, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n128s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*4, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n256s1/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*4, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n256s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*8, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n512s1/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*8, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n512s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Dense(1024)(n)
    n = LeakyReLU(alpha=0.2)(n)
    n = Dense(1,activation='sigmoid')(n)
    
    return Model(inputLayer, n)

SRGAN_disc = Discriminator(hr_shape)
SRGAN_disc.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['binary_accuracy'])
SRGAN_disc.trainable = False
SRGAN_disc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (InputLayer)              (None, 224, 224, 3)       0         
_________________________________________________________________
n64s1/c (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 64)      0         
_________________________________________________________________
n64s2/c (Conv2D)             (None, 112, 112, 64)      36928     
_________________________________________________________________
batch_normalization_3 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 112, 112, 64)      0         
_________________________________________________________________
n128s1/c (Conv2D)            (None, 112, 112, 128)     73856     
__________

/home/pratik_p_kubal/anaconda3/envs/srgan/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [7]:
def buildInceptionV3(hr_shape):
    iv3Input = Input(shape=hr_shape)
    iv3 = inception_v3.InceptionV3(weights="imagenet",input_tensor = iv3Input)
    
    return Model(iv3Input,iv3.get_layer('mixed9').output)

iv3Model = buildInceptionV3(hr_shape)
for l in iv3Model.layers:
    l.trainable = False

96116736/96112376 [==============================] - 6s 0us/step


In [8]:
iv3Model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 111, 111, 32) 96          conv2d_3[0][0]                   
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 111, 111, 32) 0           batch_normalization_10[0][0]     
__________________________________________________________________________________________________
conv2d_4 (

In [7]:
#SVG(model_to_dot(iv3, show_layer_names=True, show_shapes=True).create(prog='dot', format='svg'))

In [9]:
img_lr = Input(shape=lr_shape)
img_hr = Input(shape=hr_shape)
gen_hr = SRGAN_gen(img_lr)
validity = SRGAN_disc(gen_hr)
iv3_features = iv3Model(gen_hr)
combined = Model(img_lr, [validity, iv3_features])
combined.compile(loss=['binary_crossentropy', 'cosine_proximity'],
                              loss_weights=[1e-3, 1],
                              optimizer=optimizer)
combined.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 56, 56, 3)    0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 224, 224, 3)  1554179     input_2[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 (None, 14, 14, 1)    5219137     model_1[1][0]                    
__________________________________________________________________________________________________
model_3 (Model)                 (None, 5, 5, 2048)   15722720    model_1[1][0]                    
Total params: 22,496,036
Trainable params: 1,549,699
Non-trainable params: 20,946,337
_______________________

In [10]:
path = "./mainDataset"
dataset_name = "train"
# for the VGG feature out true labels
disc_patch = (14, 14, 1)
ni = np.sqrt(batch_size)

In [11]:
train_hr_img_list = sorted(tl.files.load_file_list(path=path+'/%s/' % (dataset_name), regx='.*.png', printable=False))
train_hr_imgs = tl.vis.read_images(train_hr_img_list[0:100], path=path+'/%s/' % (dataset_name), n_threads=32)

[TL] read 32 from ./mainDataset/train/
[TL] read 64 from ./mainDataset/train/
[TL] read 96 from ./mainDataset/train/
[TL] read 100 from ./mainDataset/train/


In [13]:
steps = len(train_hr_imgs)//batch_size
epochs = 400

In [18]:
def scale(x):
    x = x / (255. / 2.)
    x = x - 1.
    return x
    
def datagen(dev_hr_imgs,batchSize,is_testing=False):
    while(True):
        imgs_hr=[]
        imgs_lr=[]
        imgs = np.random.choice(dev_hr_imgs,batchSize)
        img_hr = tl.prepro.threading_data(imgs, fn=crop, wrg=224, hrg=224, is_random=True)
        img_lr = tl.prepro.threading_data(img_hr, fn=imresize,size=[56, 56], interp='bicubic', mode=None)
        
        imgs_hr = tl.prepro.threading_data(img_hr,fn=scale)
        imgs_lr = tl.prepro.threading_data(img_lr,fn=scale)
        
        yield imgs_hr, imgs_lr

In [19]:
datagenObj = datagen(train_hr_imgs,batch_size)

In [20]:
sample_hr,sample_lr = next(datagenObj)

In [22]:
tl.vis.save_images(sample_hr, [int(ni), int(ni)],'images/inception/'+dataset_name+'/sample_hr.png')
tl.vis.save_images(sample_lr, [int(ni), int(ni)],'images/inception/'+dataset_name+'/sample_lr.png')

In [ ]:
tensorboard = adwTensorBoard(
  log_dir='log/srgan_fixed/run0',
  histogram_freq=0,
  batch_size=batch_size,
  write_graph=True,
  write_grads=True
)
tensorboard.set_model(combined)

In [25]:
# Training SRResNet
SRGAN_disc.trainable = False
for epoch in range(100):
    print("Epoch:"+str(epoch))
    for step in tqdm_notebook(range(0,steps)):
        imgs_hr, imgs_lr = next(datagenObj)
        # The generators want the discriminators to label the generated images as real
        valid = np.ones((batch_size,14,14,1))

        # Extract ground truth image features using pre-trained VGG19 model
        image_features = iv3Model.predict(imgs_hr)

        # Train the generators
        g_loss = combined.train_on_batch(imgs_lr, [valid, image_features])
        
        
    out = SRGAN_gen.predict(sample_lr)
    tl.vis.save_images(out, [int(ni), int(ni)],'images/inception/'+dataset_name+'/train.png')
    if(epoch % 10 == 0):
        out = SRGAN_gen.predict(sample_lr)
        tl.vis.save_images(out, [int(ni), int(ni)],'images/inception/'+dataset_name+'/trainSRResNet_%d.png' % int(epoch))

Epoch:0


ResourceExhaustedError: OOM when allocating tensor with shape[16,3,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_3/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node mixed9/concat}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
# Training SRResNet
SRGAN_disc.trainable = False
for epoch in range(100):
    print("Epoch:"+str(epoch))
    for step in tqdm_notebook(range(0,steps)):
        imgs_hr, imgs_lr = next(datagenObj)
        # The generators want the discriminators to label the generated images as real
        valid = np.ones((batch_size,14,14,1))

        # Extract ground truth image features using pre-trained VGG19 model
        image_features = vggModel.predict(imgs_hr)

        # Train the generators
        g_loss = combined.train_on_batch(imgs_lr, [valid, image_features])
        
        
    out = SRGAN_gen.predict(sample_lr)
    tl.vis.save_images(out, [int(ni), int(ni)],'images/'+dataset_name+'/train.png')
    if(epoch % 10 == 0):
        out = SRGAN_gen.predict(sample_lr)
        tl.vis.save_images(out, [int(ni), int(ni)],'images/'+dataset_name+'/trainSRResNet_%d.png' % int(epoch))

In [ ]:

for epoch in range(epochs):
    print("Epoch:"+str(epoch))
    for step in tqdm_notebook(range(0,steps)):
        # Sample images and their conditioning counterparts
        imgs_hr, imgs_lr = next(datagenObj)
        # From low res. image generate high res. version
        fake_hr = SRGAN_gen.predict(imgs_lr)
        if step % 2 == 0:
            # ----------------------
            #  Train Discriminator
            # ----------------------
            
            valid = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            fake = np.random.random_sample(batch_size)*0.2

            # Train the discriminators (original images = real / generated = Fake)
            SRGAN_disc.trainable = True
            d_loss_real = SRGAN_disc.train_on_batch(imgs_hr, valid)
            d_loss_fake = SRGAN_disc.train_on_batch(fake_hr, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            SRGAN_disc.trainable = False
            
        else:
            # ------------------
            #  Train Generator
            # ------------------
            # The generators want the discriminators to label the generated images as real
            valid = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2

            # Extract ground truth image features using pre-trained VGG19 model
            image_features = vggModel.predict(imgs_hr)

            # Train the generators
            g_loss = combined.train_on_batch(imgs_lr, [valid, image_features])
        
        out = SRGAN_gen.predict(sample_lr)
        tl.vis.save_images(out, [int(ni), int(ni)],'images/'+dataset_name+'/train.png')
    #if(epoch % 100 == 0):
        #out = SRGAN_gen.predict(sample_lr)
        #tl.vis.save_images(out, [int(ni), int(ni)],'images/'+dataset_name+'/train_%d.png' % int(epoch))
    #tensorboard.on_epoch_end(epoch, {"d_mse": d_loss[0],"d_acc":d_loss[1],"g_loss":g_loss[0],"g_mse":g_loss[1]})
    #SRGAN_gen.save_weights("./checkpoints/gen.h5")
    #SRGAN_disc.save_weights("./checkpoints/disc.h5")
#tensorboard.on_train_end(None)

Epoch:0
